<a href="https://colab.research.google.com/github/MStamirski/Spaceship-Titanic/blob/main/Model_LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%cd "/content/drive/MyDrive/Colab_Notebooks/SDA_upskill/Spaceship"

In [ ]:
!pip install lightgbm

# Model with parameters

In [ ]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score

In [ ]:
model = lgb.LGBMClassifier()

In [ ]:
def objective(trial):

    params = {
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100)
    }

    optuna_model = lgb.LGBMClassifier(**params)
    optuna_model.fit(X_train, y_train)

    y_pred = optuna_model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)

    return accuracy

# Features datasets

In [ ]:
!pip install import-ipynb

In [ ]:
import import_ipynb

In [ ]:
from FeaturesEngineering import get_features, categories_one_hot_encoding, categories_target_encoding, categories_leave_one_out_encoding

In [ ]:
df = get_features('train')
df_ohe = categories_one_hot_encoding(df)
df_te = categories_target_encoding(df)
df_looe = categories_leave_one_out_encoding(df)

# Initial verification

In [ ]:
from Optimization import verify_feature_dataset, get_subsets, model_optimization, test_tuned_model

In [ ]:
acc_ohe_ini = verify_feature_dataset(df_ohe, model)

Accuracy: 78.18251533742331%


In [ ]:
acc_te_ini = verify_feature_dataset(df_te, model)

Accuracy: 78.94938650306749%


In [ ]:
acc_looe_ini = verify_feature_dataset(df_looe, model)

Accuracy: 77.03220858895705%


# Parameters tuning

In [ ]:
from optuna.samplers import TPESampler
sampler = TPESampler(seed=1)

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = get_subsets(df_ohe)
params = model_optimization(20, objective, samp=sampler)

[I 2023-04-08 19:15:18,201] A new study created in memory with name: no-name-7be131df-b924-4231-93eb-f482d693ef47
[I 2023-04-08 19:15:18,311] Trial 0 finished with value: 0.7757009345794392 and parameters: {'lambda_l1': 5.6649755830282306e-05, 'lambda_l2': 0.030403280126677572, 'num_leaves': 2, 'feature_fraction': 0.5813995435791038, 'bagging_fraction': 0.48805353449026784, 'bagging_freq': 1, 'min_child_samples': 22}. Best is trial 0 with value: 0.7757009345794392.


[LightGBM] [Warning] lambda_l1 is set=5.6649755830282306e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.6649755830282306e-05
[LightGBM] [Warning] feature_fraction is set=0.5813995435791038, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5813995435791038
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=0.030403280126677572, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.030403280126677572
[LightGBM] [Warning] bagging_fraction is set=0.48805353449026784, subsample=1.0 will be ignored. Current value: bagging_fraction=0.48805353449026784
[LightGBM] [Warning] lambda_l1 is set=1.2883876209377052e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.2883876209377052e-05
[LightGBM] [Warning] feature_fraction is set=0.6515167086419769, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6515167086419769
[LightGBM] [War

[I 2023-04-08 19:15:19,113] Trial 1 finished with value: 0.805176132278936 and parameters: {'lambda_l1': 1.2883876209377052e-05, 'lambda_l2': 3.72312200494449e-05, 'num_leaves': 139, 'feature_fraction': 0.6515167086419769, 'bagging_fraction': 0.8111317002380557, 'bagging_freq': 2, 'min_child_samples': 89}. Best is trial 1 with value: 0.805176132278936.


[LightGBM] [Warning] lambda_l1 is set=1.763958399884789e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.763958399884789e-08
[LightGBM] [Warning] feature_fraction is set=0.735213897067451, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.735213897067451
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] lambda_l2 is set=0.010819509974097813, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.010819509974097813
[LightGBM] [Warning] bagging_fraction is set=0.4842321631571403, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4842321631571403


[I 2023-04-08 19:15:21,201] Trial 2 finished with value: 0.8037383177570093 and parameters: {'lambda_l1': 1.763958399884789e-08, 'lambda_l2': 0.010819509974097813, 'num_leaves': 108, 'feature_fraction': 0.735213897067451, 'bagging_fraction': 0.4842321631571403, 'bagging_freq': 2, 'min_child_samples': 81}. Best is trial 1 with value: 0.805176132278936.


...

[LightGBM] [Warning] lambda_l1 is set=0.0002921275560269595, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0002921275560269595
[LightGBM] [Warning] feature_fraction is set=0.918791290057132, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.918791290057132
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l2 is set=0.4505582970336609, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.4505582970336609
[LightGBM] [Warning] bagging_fraction is set=0.9101690807113071, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9101690807113071


[I 2023-04-08 19:15:46,827] Trial 17 finished with value: 0.8037383177570093 and parameters: {'lambda_l1': 0.0002921275560269595, 'lambda_l2': 0.4505582970336609, 'num_leaves': 91, 'feature_fraction': 0.918791290057132, 'bagging_fraction': 0.9101690807113071, 'bagging_freq': 5, 'min_child_samples': 53}. Best is trial 3 with value: 0.8123652048885693.
[I 2023-04-08 19:15:46,961] Trial 18 finished with value: 0.8058950395398994 and parameters: {'lambda_l1': 0.012148220538081511, 'lambda_l2': 0.002353473009545225, 'num_leaves': 4, 'feature_fraction': 0.837250628796929, 'bagging_fraction': 0.8598022393299446, 'bagging_freq': 1, 'min_child_samples': 86}. Best is trial 3 with value: 0.8123652048885693.


[LightGBM] [Warning] lambda_l1 is set=0.012148220538081511, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.012148220538081511
[LightGBM] [Warning] feature_fraction is set=0.837250628796929, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.837250628796929
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=0.002353473009545225, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.002353473009545225
[LightGBM] [Warning] bagging_fraction is set=0.8598022393299446, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8598022393299446
[LightGBM] [Warning] lambda_l1 is set=0.3031698611293977, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.3031698611293977
[LightGBM] [Warning] feature_fraction is set=0.772600092505302, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.772600092505302
[LightGBM] [Warning] bagging_freq

[I 2023-04-08 19:15:47,203] Trial 19 finished with value: 0.805176132278936 and parameters: {'lambda_l1': 0.3031698611293977, 'lambda_l2': 8.221430169867247, 'num_leaves': 110, 'feature_fraction': 0.772600092505302, 'bagging_fraction': 0.772612216280748, 'bagging_freq': 4, 'min_child_samples': 65}. Best is trial 3 with value: 0.8123652048885693.



Number of finished trials: 20
Best trial:
Value: 0.8123652048885693
  Params: 
    lambda_l1: 5.180291295699627
    lambda_l2: 6.6193844201488494e-06
    num_leaves: 178
    feature_fraction: 0.9258334913776229
    bagging_fraction: 0.9367639981023084
    bagging_freq: 1
    min_child_samples: 8


In [ ]:
model = lgb.LGBMClassifier(**params)
result_ohe_tuned = test_tuned_model(model, X_train, y_train, X_test, y_test)

[LightGBM] [Warning] lambda_l1 is set=5.180291295699627, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.180291295699627
[LightGBM] [Warning] feature_fraction is set=0.9258334913776229, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9258334913776229
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=6.6193844201488494e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=6.6193844201488494e-06
[LightGBM] [Warning] bagging_fraction is set=0.9367639981023084, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9367639981023084

Accuracy after tuning: 79.12593444508337%

Classification report
              precision    recall  f1-score   support

       False       0.80      0.77      0.79       861
        True       0.78      0.81      0.80       878

    accuracy                           0.79      1739
   macro avg       0.79      0.79 

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = get_subsets(df_te)
params = model_optimization(20, objective, samp=sampler)

[I 2023-04-08 19:15:47,650] A new study created in memory with name: no-name-31964fbf-c35f-4951-bf14-21a8a2d3e070
[I 2023-04-08 19:15:47,807] Trial 0 finished with value: 0.800862688713156 and parameters: {'lambda_l1': 1.3508814484493448, 'lambda_l2': 1.7269204695427844e-07, 'num_leaves': 37, 'feature_fraction': 0.8844347732257143, 'bagging_fraction': 0.6386061021913202, 'bagging_freq': 2, 'min_child_samples': 94}. Best is trial 0 with value: 0.800862688713156.


[LightGBM] [Warning] lambda_l1 is set=1.3486297561351885e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.3486297561351885e-05
[LightGBM] [Warning] feature_fraction is set=0.9299836547234859, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9299836547234859
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] lambda_l2 is set=0.05718852932029385, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.05718852932029385
[LightGBM] [Warning] bagging_fraction is set=0.7742033242333654, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7742033242333654


[I 2023-04-08 19:15:48,116] Trial 1 finished with value: 0.800862688713156 and parameters: {'lambda_l1': 1.3486297561351885e-05, 'lambda_l2': 0.05718852932029385, 'num_leaves': 187, 'feature_fraction': 0.9299836547234859, 'bagging_fraction': 0.7742033242333654, 'bagging_freq': 6, 'min_child_samples': 38}. Best is trial 0 with value: 0.800862688713156.


[LightGBM] [Warning] lambda_l1 is set=2.6875157982766948e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.6875157982766948e-06
[LightGBM] [Warning] feature_fraction is set=0.9789040282890313, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9789040282890313
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l2 is set=1.1560482583788434, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.1560482583788434
[LightGBM] [Warning] bagging_fraction is set=0.7980648986910688, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7980648986910688


[I 2023-04-08 19:15:48,808] Trial 2 finished with value: 0.794392523364486 and parameters: {'lambda_l1': 2.6875157982766948e-06, 'lambda_l2': 1.1560482583788434, 'num_leaves': 111, 'feature_fraction': 0.9789040282890313, 'bagging_fraction': 0.7980648986910688, 'bagging_freq': 5, 'min_child_samples': 16}. Best is trial 0 with value: 0.800862688713156.


...

[LightGBM] [Warning] lambda_l1 is set=9.251468143597606e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.251468143597606e-05
[LightGBM] [Warning] feature_fraction is set=0.6772608157082901, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6772608157082901
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3
[LightGBM] [Warning] lambda_l2 is set=5.722463159817255e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5.722463159817255e-07
[LightGBM] [Warning] bagging_fraction is set=0.8637854440611954, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8637854440611954


[I 2023-04-08 19:15:55,841] Trial 18 finished with value: 0.798705966930266 and parameters: {'lambda_l1': 9.251468143597606e-05, 'lambda_l2': 5.722463159817255e-07, 'num_leaves': 123, 'feature_fraction': 0.6772608157082901, 'bagging_fraction': 0.8637854440611954, 'bagging_freq': 3, 'min_child_samples': 44}. Best is trial 14 with value: 0.8087706685837527.


[LightGBM] [Warning] lambda_l1 is set=0.0006384568018630738, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0006384568018630738
[LightGBM] [Warning] feature_fraction is set=0.5935572310318713, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5935572310318713
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] lambda_l2 is set=0.00269718929214816, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.00269718929214816
[LightGBM] [Warning] bagging_fraction is set=0.9768655891718595, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9768655891718595


[I 2023-04-08 19:15:56,114] Trial 19 finished with value: 0.807332854061826 and parameters: {'lambda_l1': 0.0006384568018630738, 'lambda_l2': 0.00269718929214816, 'num_leaves': 225, 'feature_fraction': 0.5935572310318713, 'bagging_fraction': 0.9768655891718595, 'bagging_freq': 4, 'min_child_samples': 65}. Best is trial 14 with value: 0.8087706685837527.



Number of finished trials: 20
Best trial:
Value: 0.8087706685837527
  Params: 
    lambda_l1: 0.021139466940639787
    lambda_l2: 7.846489121019732e-06
    num_leaves: 63
    feature_fraction: 0.6622556022515516
    bagging_fraction: 0.9874066124989476
    bagging_freq: 1
    min_child_samples: 83


In [ ]:
model = lgb.LGBMClassifier(**params)
result_te_tuned = test_tuned_model(model, X_train, y_train, X_test, y_test)

[LightGBM] [Warning] lambda_l1 is set=0.021139466940639787, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.021139466940639787
[LightGBM] [Warning] feature_fraction is set=0.6622556022515516, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6622556022515516
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=7.846489121019732e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.846489121019732e-06
[LightGBM] [Warning] bagging_fraction is set=0.9874066124989476, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9874066124989476

Accuracy after tuning: 78.83841288096608%

Classification report
              precision    recall  f1-score   support

       False       0.79      0.79      0.79       861
        True       0.79      0.79      0.79       878

    accuracy                           0.79      1739
   macro avg       0.79      0

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = get_subsets(df_looe)
params = model_optimization(20, objective, samp=sampler)

[I 2023-04-08 19:15:56,539] A new study created in memory with name: no-name-c2b612b9-5cb9-492e-a805-94ad636d7336
[I 2023-04-08 19:15:56,754] Trial 0 finished with value: 0.7814521926671459 and parameters: {'lambda_l1': 0.0010142899116384612, 'lambda_l2': 1.690818655548677e-07, 'num_leaves': 17, 'feature_fraction': 0.4728060734444424, 'bagging_fraction': 0.42673112712685707, 'bagging_freq': 1, 'min_child_samples': 26}. Best is trial 0 with value: 0.7814521926671459.
[I 2023-04-08 19:15:56,853] Trial 1 finished with value: 0.7915168943206327 and parameters: {'lambda_l1': 0.026115649029146128, 'lambda_l2': 0.001090066111976454, 'num_leaves': 5, 'feature_fraction': 0.4431845678136921, 'bagging_fraction': 0.9803657980001632, 'bagging_freq': 4, 'min_child_samples': 24}. Best is trial 1 with value: 0.7915168943206327.


[LightGBM] [Warning] lambda_l1 is set=0.026115649029146128, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.026115649029146128
[LightGBM] [Warning] feature_fraction is set=0.4431845678136921, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4431845678136921
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] lambda_l2 is set=0.001090066111976454, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.001090066111976454
[LightGBM] [Warning] bagging_fraction is set=0.9803657980001632, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9803657980001632
[LightGBM] [Warning] lambda_l1 is set=1.8660861996646596e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8660861996646596e-06
[LightGBM] [Warning] feature_fraction is set=0.7488153563639547, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7488153563639547
[LightGBM] [Warning] 

[I 2023-04-08 19:15:57,274] Trial 2 finished with value: 0.7828900071890726 and parameters: {'lambda_l1': 1.8660861996646596e-06, 'lambda_l2': 0.04948034130913776, 'num_leaves': 51, 'feature_fraction': 0.7488153563639547, 'bagging_fraction': 0.9820119934529874, 'bagging_freq': 6, 'min_child_samples': 28}. Best is trial 1 with value: 0.7915168943206327.


[LightGBM] [Warning] lambda_l1 is set=0.00027792488102483314, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00027792488102483314
[LightGBM] [Warning] feature_fraction is set=0.4940748367876506, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4940748367876506
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=0.003798406718422058, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.003798406718422058
[LightGBM] [Warning] bagging_fraction is set=0.4111457213064457, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4111457213064457


[I 2023-04-08 19:15:57,513] Trial 3 finished with value: 0.7828900071890726 and parameters: {'lambda_l1': 0.00027792488102483314, 'lambda_l2': 0.003798406718422058, 'num_leaves': 213, 'feature_fraction': 0.4940748367876506, 'bagging_fraction': 0.4111457213064457, 'bagging_freq': 1, 'min_child_samples': 51}. Best is trial 1 with value: 0.7915168943206327.


...

[LightGBM] [Warning] lambda_l1 is set=0.43393824018590366, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.43393824018590366
[LightGBM] [Warning] feature_fraction is set=0.6725212485489659, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6725212485489659
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] lambda_l2 is set=1.4121264626764952e-08, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.4121264626764952e-08
[LightGBM] [Warning] bagging_fraction is set=0.9310473687985822, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9310473687985822


[I 2023-04-08 19:16:07,150] Trial 18 finished with value: 0.7792954708842559 and parameters: {'lambda_l1': 0.43393824018590366, 'lambda_l2': 1.4121264626764952e-08, 'num_leaves': 44, 'feature_fraction': 0.6725212485489659, 'bagging_fraction': 0.9310473687985822, 'bagging_freq': 2, 'min_child_samples': 41}. Best is trial 11 with value: 0.7936736161035226.


[LightGBM] [Warning] lambda_l1 is set=0.794633568568987, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.794633568568987
[LightGBM] [Warning] feature_fraction is set=0.414562480994978, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.414562480994978
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4
[LightGBM] [Warning] lambda_l2 is set=0.07156452471571594, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.07156452471571594
[LightGBM] [Warning] bagging_fraction is set=0.8179053739827917, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8179053739827917


[I 2023-04-08 19:16:07,410] Trial 19 finished with value: 0.7742631200575126 and parameters: {'lambda_l1': 0.794633568568987, 'lambda_l2': 0.07156452471571594, 'num_leaves': 30, 'feature_fraction': 0.414562480994978, 'bagging_fraction': 0.8179053739827917, 'bagging_freq': 4, 'min_child_samples': 83}. Best is trial 11 with value: 0.7936736161035226.



Number of finished trials: 20
Best trial:
Value: 0.7936736161035226
  Params: 
    lambda_l1: 9.772906213555897
    lambda_l2: 3.461773950137805e-05
    num_leaves: 9
    feature_fraction: 0.6282789521745081
    bagging_fraction: 0.8815908214390611
    bagging_freq: 7
    min_child_samples: 76


In [ ]:
model = lgb.LGBMClassifier(**params)
result_looe_tuned = test_tuned_model(model, X_train, y_train, X_test, y_test)

[LightGBM] [Warning] lambda_l1 is set=9.772906213555897, reg_alpha=0.0 will be ignored. Current value: lambda_l1=9.772906213555897
[LightGBM] [Warning] feature_fraction is set=0.6282789521745081, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6282789521745081
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] lambda_l2 is set=3.461773950137805e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.461773950137805e-05
[LightGBM] [Warning] bagging_fraction is set=0.8815908214390611, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8815908214390611

Accuracy after tuning: 78.26336975273146%

Classification report
              precision    recall  f1-score   support

       False       0.80      0.75      0.77       861
        True       0.77      0.81      0.79       878

    accuracy                           0.78      1739
   macro avg       0.78      0.78   

# Save results

In [ ]:
data = []
data.append(['LightGBM', 'OHE', acc_ohe_ini, result_ohe_tuned[0], str(result_ohe_tuned[1])+" / 90"])
data.append(['LightGBM', 'TE', acc_te_ini, result_te_tuned[0], str(result_te_tuned[1])+" / 14"])
data.append(['LightGBM', 'LOOE', acc_looe_ini, result_looe_tuned[0], str(result_looe_tuned[1])+" / 14"])

In [ ]:
import csv
from os.path import exists
resfile = 'spaceship_results.csv'

In [ ]:
if exists(resfile):
  f = open(resfile, 'a')
  writer = csv.writer(f)
else:
  header = ['Model', 'Categories_encoding', 'Initial_accuracy', 'Tuned_Accuracy', 'Important_Features']
  f = open(resfile, 'w', newline='')
  writer = csv.writer(f)
  writer.writerow(header)

writer.writerows(data)
f.close()